<a href="https://colab.research.google.com/github/Anakha-s/Data-Science/blob/main/eg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
## EDA Pkgs
import pandas as pd
#Data Viz Pkg
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import numpy as np # linear algebra
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# Load Dataset
df = pd.read_csv("covid19_tweets.csv")
# Preview
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False


In [29]:
!pip install neattext
# Load Text Cleaning Package
import neattext.functions as nfx

In [30]:
# Cleaning Text
df['clean_tweet'] = df['text'].apply(nfx.remove_hashtags)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))
# Cleaning Text: Multiple WhiteSpaces
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces)
# Cleaning Text : Remove urls
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)
# Cleaning Text: Punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts)
#Remove blank rows if any.
df['clean_tweet'] .dropna(inplace=True)
# Change all the text to lower case.
df['clean_tweet']  = [entry.lower() for entry in df['clean_tweet'] ]
df[['text','clean_tweet']]

,text,clean_tweet
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey and wouldnt it have made more sense to ha...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never once claimed was a hoax we all cl...
3,@brookbanktv The one gift #COVID19 has give me...,the one gift has give me is an appreciation f...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july : media bulletin on novel
...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,thanks for nominating me for the challenge i n...
179104,2020! The year of insanity! Lol! #COVID19 http...,2020 the year of insanity lol
179105,@CTVNews A powerful painting by Juan Lucena. I...,a powerful painting by juan lucena its a trib...
179106,"More than 1,200 students test positive for #CO...",more than 1200 students test positive for at m...


In [31]:
# Tokenization
df['clean_tweet']= [word_tokenize(entry) for entry in df['clean_tweet']]
df['clean_tweet']



0         [if, i, smelled, the, scent, of, hand, sanitiz...
1         [hey, and, wouldnt, it, have, made, more, sens...
2         [trump, never, once, claimed, was, a, hoax, we...
3         [the, one, gift, has, give, me, is, an, apprec...
4                 [25, july, :, media, bulletin, on, novel]
                                ...                        
179103    [thanks, for, nominating, me, for, the, challe...
179104                 [2020, the, year, of, insanity, lol]
179105    [a, powerful, painting, by, juan, lucena, its,...
179106    [more, than, 1200, students, test, positive, f...
179107                     [i, stop, when, i, see, a, stop]
Name: clean_tweet, Length: 179108, dtype: object

In [32]:
df['clean_tweet'].head(1)

0    [if, i, smelled, the, scent, of, hand, sanitiz...
Name: clean_tweet, dtype: object

In [33]:
stop_words = set(stopwords.words('english'))

In [34]:


def remove_stopwords(words):
  return [word for word in words if  not word in stop_words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: remove_stopwords(x))


def stemming(words):
  ps=PorterStemmer()
  return [ps.stem(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: stemming(x))

def lemmatizing(words):
    lemmatizer =WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: lemmatizing(x))

def final_text(words):
     return ' '.join(words)
df['clean_tweet']=df['clean_tweet'].apply(lambda x:final_text(x))
   


In [35]:
df['clean_tweet'].head(1)

0    smell scent hand sanit today someon past would...
Name: clean_tweet, dtype: object

In [36]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 1
    elif sentiment_polarity < 0:
        sentiment_label = -1
    else:
        sentiment_label = 0
    result = {'polarity':sentiment_polarity,
              'subjectivity':sentiment_subjectivity,
              'sentiment':sentiment_label}
    return result

In [37]:
df['sentiment_results'] = df['clean_tweet'].apply(get_sentiment)
df['sentiment_results']

0         {'polarity': -0.25, 'subjectivity': 0.25, 'sen...
1         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
2         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
3         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
4         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
                                ...                        
179103    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
179104    {'polarity': 0.8, 'subjectivity': 0.7, 'sentim...
179105    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
179106    {'polarity': 0.0625, 'subjectivity': 0.5, 'sen...
179107    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
Name: sentiment_results, Length: 179108, dtype: object

In [38]:
df = df.join(pd.json_normalize(df['sentiment_results']))
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,clean_tweet,sentiment_results,polarity,subjectivity,sentiment
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,smell scent hand sanit today someon past would...,"{'polarity': -0.25, 'subjectivity': 0.25, 'sen...",-0.25,0.25,-1
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,hey wouldnt made sen player pay respect a…,"{'polarity': 0.0, 'subjectivity': 0.0, 'sentim...",0.00,0.00,0
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,trump never claim hoax claim effort to…,"{'polarity': 0.0, 'subjectivity': 0.0, 'sentim...",0.00,0.00,0


In [39]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['clean_tweet'],df['sentiment'],test_size=0.3)

In [40]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [41]:
stop_words = set(stopwords.words('english'))

Tfidf_vect = TfidfVectorizer(max_features=None)
Tfidf_vect.fit(df['clean_tweet'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'smell': 60937, 'scent': 58598, 'hand': 34583, 'sanit': 58201, 'today': 65386, 'someon': 61301, 'past': 51150, 'would': 70640, 'think': 64816, 'intox': 38384, 'that': 64590, 'hey': 35444, 'wouldnt': 70644, 'made': 43293, 'sen': 59280, 'player': 52436, 'pay': 51312, 'respect': 56547, 'trump': 66226, 'never': 47840, 'claim': 21873, 'hoax': 35816, 'effort': 28261, 'to': 65354, 'one': 49619, 'gift': 33013, 'give': 33103, 'appreci': 15279, 'simpl': 60418, 'thing': 64807, 'alway': 14272, 'around': 15557, 'me': 44578, '25': 5075, 'juli': 39655, 'medium': 44727, 'bulletin': 19601, 'novel': 48733, 'death': 25259, 'continu': 23264, 'rise': 57042, 'almost': 14150, 'bad': 16700, 'ever': 29565, 'politician': 52714, 'busi': 19732, 'want': 69270, 'chang': 21077, 'work': 70478, 'gener': 32774, 'recruit': 55691, 'specif': 61627, 'via': 68541, 'wear': 69521, 'face': 30174, 'cover': 23863, 'shop': 60026, 'includ': 37493, 'visit': 68835, 'local': 42506, 'commun': 22639, 'pharmacy': 51927, 'pray': 53275, 

In [42]:
print(Train_X_Tfidf)

  (0, 52978)	0.3424979467292472
  (0, 49788)	0.3323032467088017
  (0, 47902)	0.29519204040430624
  (0, 37614)	0.2852984597773528
  (0, 33436)	0.3068613186913098
  (0, 31132)	0.2969293934414808
  (0, 25589)	0.4274498612386925
  (0, 23887)	0.2913468911416589
  (0, 22038)	0.3921846865881805
  (1, 64257)	0.3902885442527712
  (1, 62222)	0.3581455072917731
  (1, 52932)	0.23003807953785355
  (1, 44727)	0.31086655294874843
  (1, 25071)	0.3415739097782306
  (1, 20455)	0.1688309290611526
  (1, 19601)	0.39198180779448455
  (1, 321)	0.5206003841348942
  (2, 70389)	0.27416304757404275
  (2, 58480)	0.21472269121157123
  (2, 57288)	0.31093056106520095
  (2, 55672)	0.24991963646869544
  (2, 54462)	0.267587953142641
  (2, 40301)	0.29540837300776596
  (2, 37500)	0.38033382695281504
  (2, 29799)	0.3427796733046019
  :	:
  (125372, 23555)	0.1662539624848136
  (125373, 67732)	0.2134541709414114
  (125373, 65642)	0.21073288719163663
  (125373, 55657)	0.4959669180549725
  (125373, 55374)	0.3209395832280974
 

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:

confusion_matrix(Test_Y,predictions_SVM)

In [ ]:

print(classification_report(Test_Y,predictions_SVM))

In [ ]:

from sklearn.metrics import classification_report

In [ ]:
review = """I Love stay at home at this pandemic"""
review_vector = Tfidf_vect.transform([review])
print(SVM.predict(review_vector))